## Decision Trees, Decision Boundaries, and Evaluation

### Definition:   
 A decision tree is a predictive modeling tool that is used to create a visual representation of all possible outcomes of a decision. It is a hierarchical model that consists of a set of nodes and branches, where each node represents a decision or test and each branch represents the outcome of that decision or test. Decision trees are commonly used in machine learning, data mining, and business intelligence to help with decision-making, classification, and prediction tasks.    

![Alt text](tree_example.png)  
 
 Once the decision tree is constructed, it can be used to make predictions for new data by traversing the tree from the root node down to a leaf node, where the prediction is based on the majority class or average value of the training examples in that leaf. Decision trees have several advantages, such as being interpretable, robust to outliers, and able to handle both categorical and numerical data. However, they can also be prone to overfitting, bias, and instability, and require careful tuning of their parameters and pruning techniques to avoid these issues. 
 
#### Advantages:

1.  Easy in interpretation
2.  Less data preparation
3.  Non-parametric and non-linearity. There are no significant assumptions to be fulfilled. 
4.  Highly versatile, could perform multiple roles apart from the standard predictions.

#### Disadvantages:

1.  Overfitting problem appears. 
2.  Highly time-consuming in training. 
3.  Does not consider how the recent decision affect next few stages 

### Classification: 

##### Algorithm 
The core algorithm for building decision trees called ID3 by J. R. Quinlan which employs a top-down, greedy search through the space of possible branches with no backtracking. ID3 uses Entropy and Information Gain to construct a decision tree. In ZeroR model there is no predictor, in OneR model we try to find the single best predictor, naive Bayesian includes all predictors using Bayes' rule and the independence assumptions between predictors but decision tree includes all predictors with the dependence assumptions between predictors.   		
 
##### Entropy  
A decision tree is built top-down from a root node and involves partitioning the data into subsets that contain instances with similar values (homogenous). ID3 algorithm uses entropy to calculate the homogeneity of a sample. If the sample is completely homogeneous the entropy is zero and if the sample is an equally divided it has entropy of one.    
Entropy of one attribute = $\sum_{i =1}^c - p_ilog_2(p_i)$    
Entropy of two attribute = $\sum_{c =x} - P(c)E(c)$   
  
##### Gini
Gini Index is a powerful measure of the randomness or the impurity or entropy in the values of a dataset. Gini Index aims to decrease the impurities from the root nodes (at the top of decision tree) to the leaf nodes (vertical branches down the decision tree) of a decision tree model.    
Gini = $1-\sum_{i =1}^c p_i^2$
